<a href="https://colab.research.google.com/github/FrancLis/Multivariate-Time-Series-Forecasting/blob/main/Model_Training_Selection_Evaluation_RNN_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install fast_ml
!pip install talos
!pip install kats
!pip install scipy
!pip install joblib

In [ ]:
# Seed value
# Apparently you may use different seed values at each stage
seed_value = 0

# 1. Set the `PYTHONHASHSEED` environment variable at a fixed value
import os
os.environ['PYTHONHASHSEED'] = str(seed_value)

import numpy as np
import tensorflow as tf
import random as python_random

# The below is necessary for starting Numpy generated random numbers
# in a well-defined initial state.
np.random.seed(123)

# The below is necessary for starting core Python generated random numbers
# in a well-defined state.
python_random.seed(123)

# The below set_seed() will make random number generation
# in the TensorFlow backend have a well-defined initial state.
# For further details, see:
# https://www.tensorflow.org/api_docs/python/tf/random/set_seed
tf.random.set_seed(1234)

import seaborn as sns
import pandas as pd
import talos as ta
from joblib import dump, load
from matplotlib import pyplot as plt
from fast_ml.model_development import train_valid_test_split
from tensorflow import keras
from sklearn.preprocessing import MinMaxScaler, PowerTransformer, StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, max_error, mean_absolute_error
from tensorflow.keras import Sequential, layers, callbacks
from tensorflow.keras.layers import Input, Dense, LSTM, Dropout, GRU, Bidirectional, SimpleRNN, Conv1D, MaxPooling1D, Flatten


## 1. Data Acquisition...

## 2. Data Visualization...

## 3. Data prepocessing...

## 4. Model Choice and Learning

Load and read preprocessed data

In [ ]:
with open('Preprocessed_data_PG.npy', 'rb') as f:
    X_train = np.load(f)
    y_train = np.load(f)
    X_valid = np.load(f)
    y_valid = np.load(f)
    X_test = np.load(f)
    y_test = np.load(f)

print('All shapes are: (batch, time, features)')
print('X_train.shape:', X_train.shape, 'y_train.shape:', y_train.shape)
print('X_valid.shape:', X_valid.shape, 'y_valid.shape:', y_valid.shape)
print('X_test.shape:', X_test.shape, 'y_test.shape:', y_test.shape)

All shapes are: (batch, time, features)
X_train.shape: (10496, 20, 5) y_train.shape: (10496, 5)
X_valid.shape: (1294, 20, 5) y_valid.shape: (1294, 5)
X_test.shape: (1295, 20, 5) y_test.shape: (1295, 5)


Load scaler

In [ ]:
scaler = load('MinMaxScaler_PG.joblib')

It was verified through these lines of code that the two scales were equal

In [ ]:
# Those attributes are specific of MinMaxScaler. For other scalers they might change
# if (imported_scaler.scale_ == scaler.scale_).all() and (imported_scaler.data_max_ == scaler.data_max_).all() \
        # and (imported_scale.data_min_ == scaler.data_min_).all() and (imported_scale.data_range_ == scaler.data_range_).all():
   # print("Scalers are same")

It is necessary to import the shoe ladder then subsequently the new observations that the algorithm will predict in the future which will then be reused for the next prison prediction

In [ ]:
# from talos.utils import hidden_layers

The Hidden layers function has been imported from the Talos library but has been modified according to the needs of the project

In [ ]:
def network_shape_customized(params, last_neuron, network_type):

    '''Provides the ability to include network shape in experiments. If params
    dictionary for the round contains float value for params['shapes'] then
    a linear contraction towards the last_neuron value. The higher the value,
    the fewer layers it takes to reach lesser than last_neuron.
    Supports three inbuilt shapes 'brick', 'funnel', and 'triangle'.
    params : dict
         Scan() params for a single roundself.
    last_neuron : int
         Number of neurons on the output layer in the Keras model.
    '''
    import numpy as np
    from talos.utils.exceptions import TalosParamsError

    layers = params['hidden_layers']
    shape = params['shapes']
    # network_type == 0 --> SimpleRNN
    # network_type == 1 --> GRU
    # network_type == 2 --> LSTM
    # network_type == 3 --> CONV1D
    if network_type == 3:
      first_neuron = params['first_filter']
    else:
      first_neuron = params['first_neuron']

    out = []
    n = first_neuron

    # the case where hidden_layers is zero
    if layers == 0:
        return [0]

    # the cases where an angle is applied
    if isinstance(shape, float):

        for i in range(layers):

            n *= 1 - shape

            if n > last_neuron:
                out.append(int(n))
            else:
                out.append(last_neuron)

    # the case where a rectantular shape is used
    elif shape == 'brick':
        out = [first_neuron] * layers

    elif shape == 'funnel':
        for i in range(layers + 1):
            n -= int((first_neuron - last_neuron) / layers)
            out.append(n)
        out.pop(-1)

    elif shape == 'triangle':
        out = np.linspace(first_neuron,
                          last_neuron,
                          layers+2,
                          dtype=int).tolist()

        out.pop(0)
        out.pop(-1)
        out.reverse()

    else:
        message = "'shapes' must be float or in ['funnel', 'brick', 'triangle']"
        raise TalosParamsError(message)

    return out


def hidden_layers_customized(model, params, last_neuron, network_type):
    '''HIDDEN LAYER Generator

    NOTE: 'shapes', 'first_neuron', 'dropout', and 'hidden_layers' need
    to be present in the params dictionary.

    Hidden layer generation for the cases where number
    of layers is used as a variable in the optimization process.
    Handles things in a way where any number of layers can be tried
    with matching hyperparameters.'''

    # check for the params that are required for hidden_layers

    from tensorflow.keras.layers import Dense, Dropout, Conv1D, MaxPooling1D, SimpleRNN, GRU, LSTM
    # from .network_shape import network_shape
    from talos.utils.exceptions import TalosParamsError

    if network_type == 0:
      required = ['shapes', 'first_neuron', 'dropout', 'hidden_layers', 'rnn_units']
    elif network_type == 1:
      required = ['shapes', 'first_neuron', 'dropout', 'hidden_layers', 'gru_units']
    elif network_type == 2:
      required = ['shapes', 'first_neuron', 'dropout', 'hidden_layers', 'lstm_units']
    elif network_type == 3:
      required = ['shapes', 'first_filter', 'dropout', 'hidden_layers', 'kernel_size', 'activation', 'filter']



    for param in required:
        if param not in params:
            message = "hidden_layers requires '" + param + "' in params"
            raise TalosParamsError(message)

    layer_neurons = network_shape_customized(params, last_neuron, network_type)
    # network_type == 0 --> SimpleRNN
    # network_type == 1 --> GRU
    # network_type == 2 --> LSTM
    # network_type == 3 --> CONV1D

    if network_type == 0:
        for i in range(params['hidden_layers']):
            if params['hidden_layers'] == 0:
                model.add(SimpleRNN(layer_neurons[i], return_sequences=False,))
            else:
                if i == params['hidden_layers'] - 1:
                    model.add(SimpleRNN(layer_neurons[i], return_sequences=False,))
                else:
                    model.add(SimpleRNN(layer_neurons[i], return_sequences=True,))
    elif network_type == 1:
         for i in range(params['hidden_layers']):
          model.add(GRU(layer_neurons[i], return_sequences=True))
    elif network_type == 2:
         for i in range(params['hidden_layers']):
          model.add(LSTM(layer_neurons[i], return_sequences=True))
    elif network_type == 3:
      for i in range(params['hidden_layers']):

          model.add(Conv1D(layer_neurons[i],
                          kernel_size=params.get('kernel_size'),
                           padding = 'same', activation='relu'))
          model.add(MaxPooling1D(pool_size=2, padding='same'))
          model.add(Dropout(params['dropout']))
    else:
     message = "Model not supported"
     raise TalosParamsError(message)



**Topology shapes**

Topology shapes are package-specific names where ‘brick’ assigns the
same number of neurons in each layer, ‘triangle’ decreases the number of
neurons by a constant number with each layer so that the shape resembles a
triangle, and ‘funnel’ decreases the number of neurons by the floor of the
difference between the specified number of neurons in the first layer and last
layer divided by the number of desired hidden layers, resulting in a funnel
shape.

#### 3.1 Simple Recurrent Neural Network (RNN)

In [ ]:
def SimpleRNN_fn(x_train, y_train, x_val, y_val, params):
	  # Step 1: reset the tensorflow backend session.
    tf.keras.backend.clear_session()
    # Step 2: Define the model with variable hyperparameters.
    dropout = float(params['dropout'])
    lr = float(params['lr'])
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.SimpleRNN(params['rnn_units'], return_sequences=True,
                     input_shape=(X_train.shape[1], X_train.shape[2])))
    hidden_layers_customized(model, params, 1, 0)
    model.add(tf.keras.layers.Dropout(dropout))
    model.add(tf.keras.layers.Dense(1, activation='linear',))
    
    model.compile(
        loss='mse',
        optimizer = params['optimiz'](learning_rate=lr),
        # metrics=['mse']
    )

    stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                  patience=5)
    
    history = model.fit(
        x_train, y_train, 
        epochs=params['epochs'], 
        batch_size=params['batch_size'], 
        verbose=0,
        validation_data=[x_val, y_val],
        callbacks=[stop_early], 
    )

    return history, model, 

from tensorflow.keras.optimizers import Adam, Nadam

para = {
    'batch_size': [100],  # 100, 500, 1250,
    'epochs': [2],  # 100, 200
    'shapes': ['brick'],  # <<< required
    'first_neuron': [64],  # 64,                   # <<< required
    'hidden_layers': [4],  # 2, 3                  # <<< required
    'dropout': [0, ],  # 0.25                      # <<< required
    'rnn_units': [128],  # 128                     # <<< required
    'optimiz': [Nadam, Adam],
    'lr': [0.001]
}

scan_results_SimpleRNN = ta.Scan(x=X_train,
                 y=y_train,
                 params=para,
                 model=SimpleRNN_fn,
                 experiment_name='HyperparameterTuning_SimpleRNN',
                 x_val=X_valid,
                 y_val=y_valid,
                 disable_progress_bar=False, print_params=True)

## 4. Model Selection

In [ ]:
scan_results_SimpleRNN.details

In [ ]:
r = ta.Reporting('/content/HyperparameterTuning_SimpleRNN/020722111217.csv')

# returns the results dataframe
r.data.sort_values(by=['val_loss'], ascending=True)
#r.plot_kde

# returns the highest value for 'val_fmeasure'
#r.high('mse')

# returns the number of rounds it took to find best model
#r.rounds2high()

# draws a histogram for 'val_acc'
#r.plot_hist()


In [ ]:
# get correlation for hyperparameters against a metric
r.correlate('val_loss', ['epochs', 'loss', 'val_loss'])

In [ ]:
r.plot_corr('val_loss', ['loss', 'val_loss', 'epochs', 'rnn_units', 'dropout'])

In [ ]:
# Get the best model index with highest 'val_categorical_accuracy' 
model_id = scan_results_SimpleRNN.data['val_loss'].astype('float').argmin() - 0

model_id

In [ ]:
# Clear any previous TensorFlow session.
tf.keras.backend.clear_session()

# Load the model parameters from the scanner.
from tensorflow.keras.models import model_from_json
model = model_from_json(scan_results_SimpleRNN.saved_models[model_id])
model.set_weights(scan_results_SimpleRNN.saved_weights[model_id])
model.summary()
# model.save('./best_model')

# Load that model
# my_tf_saved_model = tf.keras.models.load_model('./saved_models/my_tf_model')
# my_tf_saved_model.summary()

In [ ]:
from keras.utils.vis_utils import plot_model
plot_model(model, to_file='model.png', show_shapes=True)

In [ ]:
model_history = scan_results_SimpleRNN.round_history[model_id]

# LOSS CURVE
# Plot train loss and validation loss
def plot_loss(history):
    plt.figure(figsize=(10, 6))
    plt.plot(history['loss'])
    plt.plot(history['val_loss'])
    plt.ylabel('Loss')
    plt.xlabel('epoch')
    plt.legend(['Train loss', 'Validation loss'], loc='upper right')
    # plt.show()


plot_loss(model_history)

## 5. Model Prediction

In [ ]:
# Make prediction
def prediction(model):
    prediction = model.predict(X_test)
    return prediction


prediction_simpleRNN = prediction(model)

# prediction_simpleRNN.shape

In [ ]:
titles = [
    "Open", "High", "Low", "Close", "Adj Close",
]


s = pd.DataFrame(data=prediction_simpleRNN, index=None, columns=[f'{x}' for x in titles], dtype=None, copy=None)
print(s)
y = pd.DataFrame(data=y_test, index=None, columns=[f'{x}' for x in titles], dtype=None, copy=None)

a = 2  # number of rows
b = 3  # number of columns
c = 1  # initialize plot counter
fig = plt.figure(figsize=(12, 9))
for i in titles:
    plt.subplot(a, b, c)
    plt.title(f'{i}')
    axs = s[f'{i}'].plot(label=f'{i} pred')
    y[f'{i}'].plot(ax=axs, label=f'{i} test')
    plt.legend(loc='upper left')
    plt.xlabel('Time (day)')
    plt.ylabel('Daily water consumption ($m^3$/capita.day)')
    c = c + 1
plt.subplots_adjust(.5)
fig.suptitle('Main title', fontsize=18, y=1)
plt.tight_layout()
plt.show()

In [ ]:
# Define a function to calculate MAE and RMSE
def evaluate_prediction(predictions, actual, model_name):
    for i, j in zip(range(0, predictions.shape[1]), titles):
        rsme = np.sqrt((mean_squared_error(predictions[i], actual[i])))
        mae = mean_absolute_error(predictions[i], actual[i])
        r2 = r2_score(predictions[i], actual[i])
        # max_err = max_error(actual, predictions)
        print(model_name + ' performance of', j, 'feature:')
        print('R^2 of: {:.4f}'.format(r2))
        print('Mean Absolute Error of: {:.4f}'.format(mae))
        print('Root Mean Square Error of: {:.4f}'.format(rsme))
        # print('Max_error: {:.4f}'.format(max_err))
        print('')
    rsme = np.sqrt((mean_squared_error(predictions, actual)))
    mae = mean_absolute_error(actual, predictions)
    r2 = r2_score(actual, predictions)
    # max_err = max_error(actual, predictions)
    print(model_name + ' average performance:')
    print('R^2: {:.4f}'.format(r2))
    print('Mean Absolute Error: {:.4f}'.format(mae))
    print('Root Mean Square Error: {:.4f}'.format(rsme))
    # print('Max_error: {:.4f}'.format(max_err))
    print('')


evaluate_prediction(prediction_simpleRNN, y_test, 'LSTM')

## 5. Prediction Out of sample

In [ ]:
def predict_out_of_sample(classifier, series, n_ahead=1):
    series = np.copy(series)
    series = series[-1]
    prediction = np.zeros((n_ahead, y_train.shape[1]))
    for i in range(n_ahead):
        current_x = series[-X_train.shape[1]:]
        current_x = current_x[newaxis, :, :]
        next_pred = classifier.predict(current_x)
        prediction[i] = next_pred
        next_pred_scaled = scaler.transform(next_pred)
        series = np.concatenate((series, next_pred_scaled), axis=0)
    return prediction


In [ ]:
# Previsione test set e proiezione nel futuro di n step
prediction = predict_out_of_sample(model, X_test, 60)

print(prediction)
print(prediction.shape)

In [ ]:
from pandas.tseries.offsets import DateOffset

add_dates = [df.index[-1] + DateOffset(days=x) for x in range(0, 60 + 1)]
future_dates = pd.DataFrame(index=add_dates[1:], columns=df.columns)
df_predict = pd.DataFrame(prediction, index=future_dates[-X_test.shape[0]:].index, columns=df.columns)
df = df.append(df_predict)

In [ ]:
for i in titles:
    plt.title(f'{i}')
    plt.plot(df[f'{i}'][-200:-59], label=f'{i} pred', color='g')
    plt.plot(df[f'{i}'][-60:], label=f'{i} pred', color='r')
    plt.legend(loc='upper left')
    plt.xlabel('Time (day)')
    plt.ylabel('Daily water consumption ($m^3$/capita.day)')
    fig.suptitle('Main title', fontsize=18, y=1)
    plt.tight_layout()
    plt.show()
